## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [42]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal([28*28, 256]), name="W1")
        self.b1 = tf.Variable(tf.random.normal([256]), name="b1")
        self.W2 = tf.Variable(tf.random.normal([256, 10]), name="W2")
        self.b2 = tf.Variable(tf.random.normal([10]), name="b2")
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, (-1, 28 * 28))
        
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits      
        
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [43]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [47]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 16.940798 ; accuracy 0.63921666
epoch 1 : loss 16.870346 ; accuracy 0.64021665
epoch 2 : loss 16.8007 ; accuracy 0.64133334
epoch 3 : loss 16.731842 ; accuracy 0.6424
epoch 4 : loss 16.663761 ; accuracy 0.64356667
epoch 5 : loss 16.596441 ; accuracy 0.64486665
epoch 6 : loss 16.529871 ; accuracy 0.6457833
epoch 7 : loss 16.46403 ; accuracy 0.64683336
epoch 8 : loss 16.39891 ; accuracy 0.64788336
epoch 9 : loss 16.33451 ; accuracy 0.6487
epoch 10 : loss 16.27081 ; accuracy 0.6497
epoch 11 : loss 16.207804 ; accuracy 0.65066665
epoch 12 : loss 16.145466 ; accuracy 0.65181667
epoch 13 : loss 16.083797 ; accuracy 0.6527167
epoch 14 : loss 16.022808 ; accuracy 0.6536667
epoch 15 : loss 15.962485 ; accuracy 0.6546
epoch 16 : loss 15.902817 ; accuracy 0.65573335
epoch 17 : loss 15.843794 ; accuracy 0.65675
epoch 18 : loss 15.785415 ; accuracy 0.6576833
epoch 19 : loss 15.727662 ; accuracy 0.6587
epoch 20 : loss 15.670527 ; accuracy 0.65958333
epoch 21 : loss 15.613992 ; accurac